In [ ]:
!pip install --force-reinstall --no-deps "tensorboard==2.19.0" "pyarrow>=14.0.0,<20.0.0"

In [ ]:
!pip install --upgrade --no-deps datasets[audio]==3.6.0 transformers accelerate evaluate jiwer tensorboard gradio

In [ ]:
from datasets import load_dataset, DatasetDict, Audio
from transformers import (
    WhisperTokenizer,
    WhisperProcessor,
    WhisperFeatureExtractor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch
import evaluate

In [ ]:
model_id = 'openai/whisper-base'
out_dir = 'whisper_durian' # Thu mục lưu toàn bộ checkpoint và log sau khi fine-tune
epochs = 3
batch_size = 32 # Số lượng mẫu audio xử lí cùng lúc trên cùng 1 bước huấn luyện

In [ ]:
atc_dataset_train = load_dataset('quochuy6543/durian_audio_updated_v', split='train')
atc_dataset_valid = load_dataset('quochuy6543/durian_audio_updated_v', split='validation')

In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_id)
tokenizer = WhisperTokenizer.from_pretrained(model_id, language='vi', task='transcribe')
processor = WhisperProcessor.from_pretrained(model_id, language='vi', task='transcribe')

In [ ]:
atc_dataset_train = atc_dataset_train.cast_column('path', Audio(sampling_rate=16000))
atc_dataset_valid = atc_dataset_valid.cast_column('path', Audio(sampling_rate=16000))

In [ ]:
def prepare_dataset(batch):
    audio = batch['path']
    batch['input_features'] = feature_extractor(audio['array'], sampling_rate=audio['sampling_rate']).input_features[0]
    batch['labels'] = tokenizer(batch['text']).input_ids
    return batch

atc_dataset_train = atc_dataset_train.map(
    prepare_dataset,
    num_proc=1
)

atc_dataset_valid = atc_dataset_valid.map(
    prepare_dataset,
    num_proc=1
)

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{'input_features': feature['input_features']} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors='pt')

        label_features = [{'input_ids': feature['labels']} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors='pt')

        labels = labels_batch['input_ids'].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch['labels'] = labels

        return batch

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained(model_id)
model.generation_config.task = 'transcribe'
model.generation_config.forced_decoder_ids = None

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [ ]:
!pip install jiwer

In [ ]:
metric = evaluate.load('wer')
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {'wer': wer}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=out_dir,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=0.00001,
    warmup_steps=1000,
    bf16=False,
    fp16=True,
    num_train_epochs=epochs,
    eval_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    predict_with_generate=True,
    generation_max_length=225,
    report_to=['tensorboard'],
    load_best_model_at_end=True,
    metric_for_best_model='wer',
    greater_is_better=False,
    dataloader_num_workers=8,
    save_total_limit=2,
    lr_scheduler_type='constant',
    seed=42,
    data_seed=42
)

In [ ]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=atc_dataset_train,
    eval_dataset=atc_dataset_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()
trainer.save_model()
processor.save_pretrained(out_dir)

In [ ]:
# Trong Colab, sau khi train xong:
import shutil

# Zip toàn bộ folder model
shutil.make_archive('whisper_durian', 'zip', out_dir)

# Download file zip về máy
from google.colab import files
files.download('whisper_durian.zip')